In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# !rm -rf saved_lifter_2d_3d_model/synthetic_cabin_bw/A_Pillar_Codriver/prediction/linear_model/lightning_logs

In [3]:
import pandas as pd
import numpy as np
import torch
import lightning.pytorch as pl
import matplotlib.pyplot as plt
# import plotly
import plotly.express as px

In [4]:
import os
import torch
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from modules.lifter_2d_3d.model.linear_model.network.linear_model import BaselineModel
from modules.lifter_2d_3d.dataset.drive_and_act_keypoint_dataset import DriveAndActKeypointDataset
from modules.lifter_2d_3d.model.repnet.lit_repnet import LitRepNet
from modules.utils.visualization import generate_connection_line, get_sample_from_loader, visualize_pose
from IPython.display import display
from experiments.experiment_config import (
    DRIVE_AND_ACT_VIEWPOINTS,
    get_drive_and_act_loaders,
)
pl.seed_everything(1234)

view_point = DRIVE_AND_ACT_VIEWPOINTS[0]
constructed_loader = get_drive_and_act_loaders(view_point, subset_percentage=100)
train_loader = constructed_loader['train_loader']
val_loader = constructed_loader['val_loader']
test_loader = constructed_loader['test_loader']
all_activities = constructed_loader['all_activities']
print(
    'train_dataset', len(train_loader.dataset),
    'val_dataset', len(val_loader.dataset),
    'test_dataset', len(test_loader.dataset)
)
model_checkpoint = ModelCheckpoint(monitor='mpjpe',mode='min', save_top_k=1)
early_stopping = EarlyStopping(monitor='mpjpe', mode="min", patience=5)

# ------------
# model
# ------------
lifter_2D_3D = BaselineModel(exclude_ankle=True, exclude_knee=True)
lit_model = LitRepNet(lifter_2D_3D=lifter_2D_3D, all_activities=all_activities)
# ------------
# training
# ------------
# saved_model_path = './saved_lifter_2d_3d_model/synthetic_cabin_bw/A_Pillar_Codriver/prediction/repnet_linear_model/'
saved_model_path = f'./saved_lifter_2d_3d_model/rq3/repnet/drive_and_act/{view_point}'
if not os.path.exists(saved_model_path):
    os.makedirs(saved_model_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device', device)
# device = 'cpu'
trainer = pl.Trainer(
    # max_steps=10,
    max_epochs=100,
    callbacks=[model_checkpoint, early_stopping],
    accelerator=device,
    check_val_every_n_epoch=5,
    default_root_dir=saved_model_path,
    reload_dataloaders_every_n_epochs=1,
    # gradient_clip_val=1.0
)
trainer.fit(lit_model, train_loader, val_loader)

Global seed set to 1234
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


train_dataset 22494 val_dataset 6157 test_dataset 10959
device cuda


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | lifter_2D_3D  | BaselineModel      | 4.3 M 
1 | camera_net    | CameraNet          | 4.0 M 
2 | generator     | RepNet             | 8.3 M 
3 | discriminator | DiscriminatorModel | 89.2 K
-----------------------------------------------------
8.4 M     Trainable params
0         Non-trainable params
8.4 M     Total params
33.650    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

check #0
val MPJPE from: 128 samples : 2496.199131011963
val P-MPJPE from: 128 samples : 2243.5804408129297
activity_macro_mpjpe 2496.685028076172
activity_macro_procrusted_mpjpe 2230.180259205083


...

check #14
training loss from 1755 batches:
d_loss = 1.4906333393181153
g_loss = -0.7561401446999987
c_loss = 0.03114469319454625
pose_2d_loss = 0.14156741803867526
total_g_loss = -0.5834280327344552
val MPJPE from: 6157 samples : 333.6929380893707
val P-MPJPE from: 6157 samples : 216.75479117839683
activity_macro_mpjpe 345.3400731086731
activity_macro_procrusted_mpjpe 217.14487390030797
current learning rate: [1.3527595427905592e-05] [1.3527595427905592e-05]
current learning rate: [1.2851215656510312e-05] [1.2851215656510312e-05]
current learning rate: [1.2208654873684796e-05] [1.2208654873684796e-05]
current learning rate: [1.1598222130000555e-05] [1.1598222130000555e-05]
current learning rate: [1.1018311023500527e-05] [1.1018311023500527e-05]


Validation: 0it [00:00, ?it/s]

check #15
training loss from 1755 batches:
d_loss = 2.135879768129767
g_loss = -0.9082101488385106
c_loss = 0.017036506270146013
pose_2d_loss = 0.10943376647780763
total_g_loss = -0.7817398767865281
val MPJPE from: 6157 samples : 456.8707346916199
val P-MPJPE from: 6157 samples : 328.42603671538603
activity_macro_mpjpe 462.32226490974426
activity_macro_procrusted_mpjpe 326.53493450079316
current learning rate: [1.04673954723255e-05] [1.04673954723255e-05]
current learning rate: [9.944025698709225e-06] [9.944025698709225e-06]
current learning rate: [9.446824413773763e-06] [9.446824413773763e-06]
current learning rate: [8.974483193085074e-06] [8.974483193085074e-06]
current learning rate: [8.52575903343082e-06] [8.52575903343082e-06]


Validation: 0it [00:00, ?it/s]

check #16
training loss from 1755 batches:
d_loss = 4.547185775425359
g_loss = -0.9996155461015185
c_loss = 0.01209084473078025
pose_2d_loss = 0.017570384301277053
total_g_loss = -0.9699543167383243
val MPJPE from: 6157 samples : 434.4000518321991
val P-MPJPE from: 6157 samples : 270.83548452431603
activity_macro_mpjpe 446.12154364585876
activity_macro_procrusted_mpjpe 272.9810631707773
current learning rate: [8.09947108175928e-06] [8.09947108175928e-06]


In [5]:
best_checkpoint_path = model_checkpoint.best_model_path
trainer.test(ckpt_path=best_checkpoint_path, dataloaders=test_loader)

Restoring states from the checkpoint path at saved_lifter_2d_3d_model/rq3/repnet/drive_and_act/a_column_co_driver/lightning_logs/version_1/checkpoints/epoch=54-step=38610.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at saved_lifter_2d_3d_model/rq3/repnet/drive_and_act/a_column_co_driver/lightning_logs/version_1/checkpoints/epoch=54-step=38610.ckpt


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   activity_macro_mpjpe    │     315.2385950088501     │
│           mpjpe           │     311.2344443798065     │
│  p_activity_macro_mpjpe   │     240.8603526046069     │
│          p_mpjpe          │    235.10867703403653     │
└───────────────────────────┴───────────────────────────┘

[{'mpjpe': 311.2344443798065,
  'p_mpjpe': 235.10867703403653,
  'activity_macro_mpjpe': 315.2385950088501,
  'p_activity_macro_mpjpe': 240.8603526046069}]

In [6]:
img = plt.imread(f'/root/data/processed/drive_and_act/inner_mirror/images/train/{item["filenames"]}')
fig, ax = plt.subplots()
ax.imshow(img)

ax.scatter(item['pose_2d'][:,0] * item['scale_factor'][0] + item['root_2d'][0], item['pose_2d'][:,1] * item['scale_factor'][1] + item['root_2d'][1])

NameError: name 'item' is not defined

In [ ]:
from pathlib import Path
view_point = 'inner_mirror'
from modules.utils.visualization import (
    plot_samples
)

plot_samples(
    Path(f'/root/data/processed/drive_and_act/') / view_point,
    lit_model,
    test_loader,
    'train',
    img_figsize=(20/2, 10/2),
    plot_figsize=(20.5/2, 10/2),
    sample_indices=[500],
    # sample_indices=[500],
    is_plot_gt_skeleton=False
)